<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/classText/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация текста. Анализ тональности текста


## Импортируем и доставляем библиотеки

In [1]:
!wget -O imdb.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vrQ5czMHoO3pEnmofFskymXMkq_u1dPc"
!unzip imdb.zip
!pip -q install eli5
!pip -q install spacy
!python -m spacy download en_core_web_sm

Archive:  imdb.zip
replace test.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.tsv                
  inflating: train.tsv               
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 559 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import numpy as np
import pandas as pd
import pickle
import re
import spacy
import eli5
import warnings

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from tqdm import tqdm

warnings.filterwarnings('ignore')

tqdm.pandas()

## Подгружаем словари для замены в тексте

In [3]:
#@title Подгружаем словари 
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may  not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great",
"mst3k": "Mystery Science Theater 3000"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

## Дополнительные функции

In [10]:
def replase_words(text,dict_):
  """
    Функция для замены на значения из словарей
  """
  output = ''
  for word in text.split(' '):
      word = word.strip()
      if word in dict_.keys(): 
          output += ' ' + dict_[word]
      else:
          output += ' ' + word
  return output

def clean_text(text):
  """
    Функция для предобработки текста
  """
  #  Удалим @user из всех твитов
  text = re.sub("@[\w]*","",text)
  # Заменим эмодзи на соответствующие им слова.
  text = replase_words(text, emoticon_dict)
  # Заменим сокращения на их полные формы
  text = replase_words(text, apostrophe_dict)
  text = replase_words(text, short_word_dict)

  # Заменим пунктуацию на пробелы.
  text = re.sub("[^\w\s]"," ",text)
  # Заменим спецсимволы на пробелы.
  return re.sub("[^a-zA-Z0-9\_]"," ",text)

def eval_model(vectorizer, classifier, test_df, train_df, name_fit_field, name_field_check):
  """
    Функция для создании модели, предсказания и оценки
  """
  model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
  ])

  model.fit(train_df[name_fit_field], train_df[name_field_check])

  preds = model.predict(test_df[name_fit_field])
  print('Test accuracy = {:.2%}'.format(accuracy_score(train_df[name_field_check], preds)))

def lemmatize_text(text):
    """
      Лемматизация текста
    """
    doc = nlp(text)
    tokens = [token.lemma_.strip() for token in doc]
    return " ".join(tokens)

def save_model(name, model):
  """
    Функция для сохранения модели
  """
  with open(name, 'wb') as f:
    pickle.dump(model, f)

def load_model(name):
  """
    Функция для загрузки модели
  """
  f = open(name, 'rb')
  return pickle.load(f)

def tagging_text(text):
  """
    Функция для тэггирования
  """
  doc = nlp(text)
  tokens=[token.ent_type_.strip() if token.ent_type_ != "" else token.text.strip() for token in doc]

  # Удалим повотрояющиеся токены.
  text = [tokens[i] for i in range(1, len(tokens)) if tokens[i] != tokens[i-1] ]
  return " ".join(text)


## Подготавливаем данные

In [5]:
train_df = pd.read_csv("train.tsv", delimiter="\t")
test_df = pd.read_csv("test.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


In [6]:
train_df['preprocessing'] = train_df['review'].apply(lambda x: clean_text(x))
test_df['preprocessing'] = test_df['review'].apply(lambda x: clean_text(x))

## 1. Посмотрим на токены, если будут мусорные добавим их в стоп слова и обучим занаво


In [7]:
#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }

positive_words = 'love', 'great', 'best', 'wonderful' #@param {type:"raw"}
negative_words = 'worst', 'awful', '1/10', 'crap' #@param {type:"raw"}

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))
if accuracy > 0.71:
    from IPython.display import Image, display
    display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

Test accuracy = 66.73%


In [11]:
# Оценим что получилось
vectorizer = CountVectorizer()
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'preprocessing', 'is_positive')

Test accuracy = 50.08%


In [12]:
# посмотрим на веса в тексте
eli5.show_weights(classifier, vec=vectorizer, top=50)

Weight?,Feature
+1.604,superb
+1.526,funniest
+1.518,wonderfully
+1.515,rare
+1.480,surprisingly
+1.435,refreshing
+1.396,enjoyable
+1.315,incredible
+1.298,perfect
+1.253,subtle


### Посмотрим на конкретные примеры его работы:



In [13]:
print('Positive' if test_df['is_positive'].iloc[1] else 'Negative')
eli5.show_prediction(classifier, test_df['preprocessing'].iloc[1], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Positive


In [14]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(classifier, test_df['preprocessing'].iloc[6], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


## 2. Проверьте изменилось ли качество при лемматизации/и без неё



In [18]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [19]:
train_df['lem_preprocessing'] = train_df['preprocessing'].progress_apply(lambda x: lemmatize_text(x))
test_df['lem_preprocessing'] = test_df['preprocessing'].progress_apply(lambda x: lemmatize_text(x))

100%|██████████| 25000/25000 [12:51<00:00, 32.42it/s]


In [20]:
#@title Проверим что получится после лемматизации
vectorizer = CountVectorizer()
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'lem_preprocessing', 'is_positive')

Test accuracy = 49.76%


In [21]:
#@title Проверим, что будет если CountVectorizer построим на биграммах.
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20000, analyzer='word')
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'lem_preprocessing', 'is_positive')

Test accuracy = 50.00%


In [22]:
#@title Проверим, что будет если CountVectorizer построим на триграммах.
vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=20000, analyzer='word')
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'lem_preprocessing', 'is_positive')

Test accuracy = 49.98%


#### **Вывод**:
При лемматизации модель отрабатывает не очень хорошо (49.76%), хотя без него модель отработала получше (не сказать что на много) - 50.08%. Были попытки использовать биграмм - 50.00% и триграмм - 49.98%.

## 3. Замените все токены которые принадлежат сущностям на их тег. Проверьте изменилось ли качество после этого

In [23]:
#@title Назначаем тегги
train_df['tagg_lem_preprocessing'] = train_df['lem_preprocessing'].progress_apply(lambda x: tagging_text(x))
test_df['tagg_lem_preprocessing'] = test_df['lem_preprocessing'].progress_apply(lambda x: tagging_text(x))

100%|██████████| 25000/25000 [12:58<00:00, 32.12it/s]


In [24]:
#@title Проверим что получится после теггирования
vectorizer = CountVectorizer()
classifier = LogisticRegression()


eval_model(vectorizer, classifier, test_df, train_df, 'tagg_lem_preprocessing', 'is_positive')

Test accuracy = 49.82%


In [25]:
#@title Проверим, что будет если CountVectorizer построим на биграммах.
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20000, analyzer='word')
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'tagg_lem_preprocessing', 'is_positive')

Test accuracy = 50.14%


In [26]:
#@title Проверим, что будет если CountVectorizer построим на биграммах.
vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=20000, analyzer='word')
classifier = LogisticRegression()

eval_model(vectorizer, classifier, test_df, train_df, 'tagg_lem_preprocessing', 'is_positive')

Test accuracy = 49.93%


#### **Вывод**:
При теггировании модель отрабатывает не очень хорошо (49.82%), хотя без него модель отработала получше (не сказать что на много) - 50.08%. Были попытки использовать биграмм - 50.14% и триграмм - 49.93%. Которые улучшили результат с лемматизацией, и только биграмм показал лучше результат.

## Сохраним модель

In [27]:
save_model('train_df.pkl', train_df)
save_model('test_df.pkl', test_df)